# Projet Big Data: Système de gestion d’emprunt dans une librairie

### 0. Création des données

In [173]:
from pyspark.context import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql import GroupedData
from pyspark.sql import DataFrame 
from pyspark.sql import Column
from pyspark.sql import Row 
from pyspark.sql import DataFrameNaFunctions 
from pyspark.sql import DataFrameStatFunctions 
from pyspark.sql import types 
from pyspark.sql import Window

In [6]:
spark=SparkSession.builder.appName('librairie').getOrCreate()
sc=spark.sparkContext
sqlContext = SQLContext(sc)

In [171]:
borrow=spark.createDataFrame(
    [
        ('S15', '0003','02-01-2020','01-02-2020'), # create your data here, be consistent in the types.
        ('S15', '0002','13-06-2020','null'),
        ('S15', '0001','13-06-2020','13-10-2020'),
        ('S16', '0002','24-01-2020','24-01-2020'),
        ('S17', '0001','12-04-2020','01-07-2020')
    ],
    ['sid', 'bid','checkout_time','return_time'] # add your columns label here
)
book=spark.createDataFrame(
    [
        ('0001',"L'existentialisme est un humanisme",'Philosophie'), # create your data here, be consistent in the types.
        ('0002',"Huis clos. Suivi de Les Mouches",'Philosophie'),
        ('0003','Mignonne allons voir si la rose','Poeme'),
        ('0004','Les Amours','Poème')],
    ['bid','title','category'] # add your columns label here
)
student=spark.createDataFrame(
    [
        ('S15', 'toto','Math'),  
        ('S16', 'popo','Eco'),
        ('S17', 'fofo','Mécanique')],
    ['sid', 'sname','dept']
)
Author=spark.createDataFrame(
    [
        ('07890', 'Jean Paul Sartre'),  
        ('05678', 'Pierre de Ronsard'),],
    ['aid', 'name']
)
write=spark.createDataFrame(
    [
        ('07890', '0001'),  
        ('07890', '0002'),
        ('05678', '0003'),  
        ('05678', '0003'),],
    ['aid', 'bid']
)

### 1-Trouver les titres de tous les livres que l'étudiant sid='S15' a emprunté.

In [94]:
#DSL
borrow.filter(borrow.sid=='S15').join(book, borrow.bid==book.bid).select("sid","title").show()

+---+--------------------+
|sid|               title|
+---+--------------------+
|S15|Huis clos. Suivi ...|
|S15|Mignonne allons v...|
|S15|L'existentialisme...|
+---+--------------------+



In [177]:
#SQL
borrow.createOrReplaceTempView("borrow_view")
book.createOrReplaceTempView("book_view")
spark.sql("""select sid,title
            from book_view
            join borrow_view
            on book_view.bid = borrow_view.bid
            where borrow_view.sid = 'S15' """).show()

+---+--------------------+
|sid|               title|
+---+--------------------+
|S15|Huis clos. Suivi ...|
|S15|Mignonne allons v...|
|S15|L'existentialisme...|
+---+--------------------+



### 2-Trouver les titres de tous les livres qui n'ont jamais été empruntés par un étudiant.

In [99]:
#DSL
book.join(borrow, borrow.bid==book.bid,'anti').select("title").show()

+----------+
|     title|
+----------+
|Les Amours|
+----------+



In [178]:
#SQL
spark.sql("""select title 
            from book_view
            left join borrow_view
            on book_view.bid = borrow_view.bid
            where borrow_view.bid is null""").show()

+----------+
|     title|
+----------+
|Les Amours|
+----------+



### 3-Trouver tous les étudiants qui ont emprunté le livre bid=’0002’

In [107]:
#DSL
borrow.filter(borrow.bid=='0002').join(student, borrow.sid==student.sid).select("bid","sname").show()

+----+-----+
| bid|sname|
+----+-----+
|0002| popo|
|0002| toto|
+----+-----+



In [180]:
#SQL
student.createOrReplaceTempView("student_view")
spark.sql("""select bid,sname
            from student_view
            join borrow_view
            on student_view.sid = borrow_view.sid
            where borrow_view.bid = '0002' """).show()

+----+-----+
| bid|sname|
+----+-----+
|0002| popo|
|0002| toto|
+----+-----+



### 4-Trouver les titres de tous les livres empruntés par des étudiants en Mécanique (département Mécanique)

In [172]:
#DSL
student.filter(student.dept=="Mécanique").join(borrow, borrow.sid==student.sid).join(book, borrow.bid==book.bid).select("title").show()

+--------------------+
|               title|
+--------------------+
|L'existentialisme...|
+--------------------+



In [181]:
#SQL
spark.sql("""select title
            from book_view
            join borrow_view
            on book_view.bid = borrow_view.bid
            join student_view
            on borrow_view.sid = Student_view.sid
            where student_view.dept = 'Mécanique' """).show()

+--------------------+
|               title|
+--------------------+
|L'existentialisme...|
+--------------------+



### 5-Trouver les étudiants qui n’ont jamais emprunté de livre. 

In [108]:
#DSL
student.join(borrow, borrow.sid==student.sid,'anti').select("sname").show()

+-----+
|sname|
+-----+
+-----+



In [183]:
#SQL
spark.sql("""select sname
            from student_view
            left join borrow_view
            on student_view.sid = borrow_view.sid
            where borrow_view.sid = 'null'""").show()

+-----+
|sname|
+-----+
+-----+



### 6- Déterminer l’auteur qui a écrit le plus de livres. 

In [185]:
#DSL
write=write.dropDuplicates()
df=write.groupBy(write.aid).agg(
    F.count(write.aid).alias('nombre'))
df.join(Author, df.aid==Author.aid).select("name",'nombre').show()

+-----------------+------+
|             name|nombre|
+-----------------+------+
|Pierre de Ronsard|     1|
| Jean Paul Sartre|     2|
+-----------------+------+



In [186]:
#QSL
write.createOrReplaceTempView("write_view")
Author.createOrReplaceTempView("author_view")
spark.sql("""select name , count(bid) as nombre
            from author_view
            join write_view
            on author_view.aid = write_view.aid
            group by name""").show()

+-----------------+------+
|             name|nombre|
+-----------------+------+
| Jean Paul Sartre|     2|
|Pierre de Ronsard|     1|
+-----------------+------+



### 7- Déterminer les personnes qui n’ont pas encore rendu les livres.

In [137]:
#DSL
borrow.filter(borrow.return_time =='null').join(student, borrow.sid==student.sid).select("sname").show()

+-----+
|sname|
+-----+
| toto|
+-----+



In [188]:
#SQL
spark.sql("""select sname
            from student_view
            join borrow_view
            on student_view.sid = borrow_view.sid
            where borrow_view.return_time = 'null' """).show()

+-----+
|sname|
+-----+
| toto|
+-----+



### 8- Créer une nouvelle colonne dans la table borrow qui prend la valeur 1, si la durée d'emprunt est supérieur à 3 mois,  sinon 0.  (utiliser la fonction spark to_date) par la suite faire un export des données en CSV. dans un répertoire nommé contention

In [164]:
tab=spark.sql("""
  SELECT TO_DATE(CAST(UNIX_TIMESTAMP(checkout_time, 'dd-MM-yyyy') AS TIMESTAMP)) AS checkout_date,TO_DATE(CAST(UNIX_TIMESTAMP(return_time, 'dd-MM-yyyy') AS TIMESTAMP)) AS return_date,bid,sid from borrow_view"""
)
tab=tab.withColumn("duree",F.datediff( "return_date","checkout_date"))
tab=tab.withColumn('Return_status', F.when(F.col('duree') > 120, "1").otherwise("0"))
cond = [borrow.sid==tab.sid , borrow.bid==tab.bid]
borrow.join(tab, cond).select(borrow.sid,borrow.bid, 'checkout_date','return_date','Return_status').show()

+---+----+-------------+-----------+-------------+
|sid| bid|checkout_date|return_date|Return_status|
+---+----+-------------+-----------+-------------+
|S15|0001|   2020-06-13| 2020-10-13|            1|
|S17|0001|   2020-04-12| 2020-07-01|            0|
|S15|0003|   2020-01-02| 2020-02-01|            0|
|S16|0002|   2020-01-24| 2020-01-24|            0|
|S15|0002|   2020-06-13|       null|            0|
+---+----+-------------+-----------+-------------+



In [175]:
borrow_csv=borrow.join(tab, cond).select(borrow.sid,borrow.bid, 'checkout_date','return_date','Return_status')
borrow_csv.toPandas().to_csv("borrow_csv.csv",index=False)

### 9-déterminer les livres qui n’ont jamais été empruntés. 

In [169]:
#DSL
book.join(borrow, borrow.bid==book.bid,'anti').select("title").show()

+----------+
|     title|
+----------+
|Les Amours|
+----------+



In [189]:
#SQL
spark.sql("""select title
            from book_view
            left join borrow_view
            on book_view.bid = borrow_view.bid
            where borrow_view.bid is null""").show()

+----------+
|     title|
+----------+
|Les Amours|
+----------+

